<a href="https://colab.research.google.com/github/ignaciomasari/PSB/blob/main/Biomass_EfficientNet_MixedInput2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#!git clone https://github.com/ignaciomasari/PSB
#import sys
#sys.path.append('/content/PSB')
#from mylib import *

import os 
import zipfile 
#!pip install tensorflow==2.1.0
#!pip install keras==2.3.1
import tensorflow as tf 
import tensorflow.keras
from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import keras


In [2]:
#!pip install efficientnet

In [3]:
from google.colab import drive
def load_images_from_folder(folder,h,w):
    images = []
    names = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(cv2.resize(img,(h,w)))
            #images.append(img)
            pos = filename.find(".")
            names.append(int(filename[:pos]))
    return [names,images]
	
def mount_data(folder,h,w):
  drive.mount('/content/drive')

  #folder2 = folder + '/concat3bis'
  folder2 = folder + '/S0'
  images = load_images_from_folder(folder2,h,w)

  harvest_dict = {}
  with open(folder + "/harvest.txt") as tsv:
      for line in csv.reader(tsv, delimiter="\t"):
          if line[0] =='pot':
              continue
          harvest_dict[int(line[0])] = [float(str.replace(line[5],',','.')) , int(line[14])]  
        
  stem_dict = {}
  with open(folder + "/stem.txt") as tsv:
      for line in csv.reader(tsv, delimiter="\t"):
          if line[0] =='Pot':
              continue
          stem_dict[int(line[0])] = [int(line[1]),int(line[2])]                  

  
  VNIR_dict = {}
  with open(folder + "/VNIR.txt") as tsv:
      for line in csv.reader(tsv, delimiter="\t"):
          if line[0] =='Pot':
              continue
          pot = int(line[0])
          VNIR_dict[pot] = []
          for i in range(1,len(line) - 1):              
              VNIR_dict[pot].append(float(line[i]))

  FW = []
  meta_data = []   
  data = [images[1],meta_data,FW]

  for p in images[0]:
      if p in harvest_dict.keys():
          FW.append(harvest_dict[p][0])
          meta_data.append([harvest_dict[p][1]])#,stem_dict[p][0],stem_dict[p][1]])
          #meta_data[-1].extend(VNIR_dict[p])

  return data  

def Divide_in_classes(Y_orig,Y_orig2, interval):

  classes = math.ceil(max([max(Y_orig),max(Y_orig2)])/interval)
  n = Y_orig.shape[0]
  Y_cl = np.zeros([n,classes])
  for y in range(n):
    Y_cl[y,math.floor(Y_orig[y] / interval)] = 1

  n = Y_orig2.shape[0]
  Y_cl2 = np.zeros([n,classes])
  for y in range(n):
    Y_cl2[y,math.floor(Y_orig2[y] / interval)] = 1

  return Y_cl,Y_cl2,classes

In [4]:
#PARAMETERS
dropout = 0.25
last_layer = 4
last_layer2 = 4
trainable_layers = 150
lr0 = 5e-4
size = 224
dense = 1024

#import efficientnet.keras as efn

In [5]:
data = mount_data("/content/drive/My Drive/PSB/Biomass-phenovision",size,size*1)

X = np.array(data[0][:][:])
X2 = np.array(data[1][:][:])
Y = np.array(data[2][:][:])


X_train_orig, X_test_orig, X2_train_orig, X2_test_orig, Y_train_orig, Y_test_orig = train_test_split(X, X2, Y, test_size = 0.2, random_state = 0)

Mounted at /content/drive


In [6]:
# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape & normalize
cs = MinMaxScaler()
X2_train = cs.fit_transform(np.reshape(X2_train_orig.T,[X2_train_orig.shape[0],X2_train_orig.shape[1]]))
X2_test = cs.transform(np.reshape(X2_test_orig.T,[X2_test_orig.shape[0],X2_test_orig.shape[1]]))
Y_train = np.reshape(Y_train_orig.T,[Y_train_orig.shape[0],1])
Y_test = np.reshape(Y_test_orig.T,[Y_test_orig.shape[0],1])

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("X2_train shape: " + str(X2_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("X2_test shape: " + str(X2_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 140
number of test examples = 36
X_train shape: (140, 224, 224, 3)
X2_train shape: (140, 1)
Y_train shape: (140, 1)
X_test shape: (36, 224, 224, 3)
X2_test shape: (36, 1)
Y_test shape: (36, 1)


In [7]:
#print ("number of training examples = " + str(X_train.shape[0]))
#print ("number of test examples = " + str(X_test.shape[0]))
#print ("X_train shape: " + str(X_train.shape))
#print ("Y_train shape: " + str(Y_train.shape))
#print ("X_test shape: " + str(X_test.shape))
#print ("Y_test shape: " + str(Y_test.shape))

In [8]:
size = 224
from keras.models import model_from_json
import os
#from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import VGG19
base_model = VGG19(input_shape = (size * 1, size, 3), include_top = False, weights = 'imagenet')


80142336/80134624 [==============================] - 3s 0us/step


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
def create_mlp(dim):
	# define our MLP network
	model = Sequential()
	model.add(Dense(1, input_dim=dim, activation="relu"))
	model.add(Dense(last_layer, activation="relu"))
	
	# return our model
	return model

mlp = create_mlp(X2_train.shape[1])

In [10]:
mlp.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 2         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 8         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [25]:
for layer in base_model.layers:
    layer.trainable = False

In [26]:
from tensorflow.python.keras.layers import Dense, Flatten, Dropout

x = base_model.output
x = Flatten()(x)
x = Dense(dense, activation="relu")(x)
x = Dropout(dropout)(x)
#predictions = Dense(last_layer, activation="relu")(x)
predictions = Dense(1, activation="linear")(x)
cnn_model = tf.keras.models.Model(base_model.input,predictions)
cnn_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [13]:
from tensorflow.keras.layers import concatenate

#x = concatenate([mlp.output,cnn_model.output])
#x = Dense(last_layer2, activation="relu")(x)
#predictions = Dense(1, activation="linear")(x)

#model_final = tf.keras.models.Model(inputs=[mlp.input, cnn_model.input],outputs=predictions)

In [31]:
model_final = cnn_model

opt = Adam(lr=lr0, decay=2e-3 / 20)
model_final.compile(optimizer = opt, loss = "mean_absolute_percentage_error")
#keras.backend.set_value(model_final.optimizer.learning_rate,0.001)
#model_final.fit(x = [X2_train,X_train], y = Y_train, epochs = 10, batch_size = 8)
model_final.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 8)




Epoch 1/10
18/18 [==============================] - 3s 105ms/step - loss: 720.4263
Epoch 2/10
18/18 [==============================] - 2s 85ms/step - loss: 25.9095
Epoch 3/10
18/18 [==============================] - 2s 84ms/step - loss: 19.4944
Epoch 4/10
18/18 [==============================] - 2s 84ms/step - loss: 31.0962
Epoch 5/10
18/18 [==============================] - 2s 84ms/step - loss: 16.5935
Epoch 6/10
18/18 [==============================] - 2s 84ms/step - loss: 19.6778
Epoch 7/10
18/18 [==============================] - 2s 84ms/step - loss: 27.4326
Epoch 8/10
18/18 [==============================] - 2s 84ms/step - loss: 18.2067
Epoch 9/10
18/18 [==============================] - 2s 84ms/step - loss: 16.3091
Epoch 10/10
18/18 [==============================] - 2s 84ms/step - loss: 17.9045


In [32]:
for layer in range(len(model_final.layers)):
   if layer < (len(model_final.layers) - trainable_layers):
    model_final.layers[layer].trainable = False
   else:
    model_final.layers[layer].trainable = True

In [33]:
#preds = model_final.evaluate(x = [X2_test,X_test], y = Y_test)
preds = model_final.evaluate(x = X_test, y = Y_test)
### END CODE HERE ###
print()
print ("Loss = " + str(preds))

2/2 [==============================] - 0s 18ms/step - loss: 11.2389

Loss = 11.23891544342041


In [34]:
#model_final.fit(x = [X2_train, X_train], y = Y_train, epochs = 10, batch_size = 16)
model_final.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 16)
#keras.backend.set_value(model_final.optimizer.learning_rate,0.00005)
#model_final.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 16)
#keras.backend.set_value(model_final.optimizer.learning_rate,0.00001)
#model_final.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 16)

Epoch 1/10
9/9 [==============================] - 2s 196ms/step - loss: 22.4491
Epoch 2/10
9/9 [==============================] - 1s 129ms/step - loss: 18.7588
Epoch 3/10
9/9 [==============================] - 1s 128ms/step - loss: 12.3617
Epoch 4/10
9/9 [==============================] - 1s 128ms/step - loss: 12.0290
Epoch 5/10
9/9 [==============================] - 1s 128ms/step - loss: 14.6330
Epoch 6/10
9/9 [==============================] - 1s 128ms/step - loss: 11.5716
Epoch 7/10
9/9 [==============================] - 1s 128ms/step - loss: 14.0517
Epoch 8/10
9/9 [==============================] - 1s 130ms/step - loss: 12.4260
Epoch 9/10
9/9 [==============================] - 1s 129ms/step - loss: 15.6541
Epoch 10/10
9/9 [==============================] - 1s 128ms/step - loss: 14.3438


In [43]:
#Y_pred = model_final.predict([X2_test,X_test])
Y_pred = model_final.predict(X_test)
#print(Y_pred)

In [36]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(Y_pred, Y_test)
r_sq = model.score(Y_pred, Y_test)
print('coefficient of determination:', r_sq)

coefficient of determination: 0.9351311236339088


In [37]:
#preds = model_final.evaluate(x = [X2_test, X_test], y = Y_test)
preds = model_final.evaluate(x = X_test, y = Y_test)

print()
print ("Loss = " + str(preds))

2/2 [==============================] - 0s 21ms/step - loss: 16.8091

Loss = 16.80908203125


In [44]:
import scipy

print(Y_pred.shape)
r_sq,p = scipy.stats.pearsonr(Y_pred.reshape((Y_pred.shape[0],)), Y_test.reshape((Y_test.shape[0],)))
print('Pearson\'s coefficient of determination:', r_sq)

(36, 1)
Pearson's coefficient of determination: 0.9664915873591162


In [41]:
#keras.backend.set_value(model_final.optimizer.learning_rate,0.00003)
#model_final.fit(x = [X2_train, X_train], y = Y_train, epochs = 10, batch_size = 16)
model_final.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 16)

Epoch 1/10
9/9 [==============================] - 1s 128ms/step - loss: 12.2716
Epoch 2/10
9/9 [==============================] - 1s 130ms/step - loss: 11.9642
Epoch 3/10
9/9 [==============================] - 1s 130ms/step - loss: 11.9507
Epoch 4/10
9/9 [==============================] - 1s 130ms/step - loss: 13.0050
Epoch 5/10
9/9 [==============================] - 1s 129ms/step - loss: 11.2348
Epoch 6/10
9/9 [==============================] - 1s 129ms/step - loss: 11.0890
Epoch 7/10
9/9 [==============================] - 1s 129ms/step - loss: 16.1014
Epoch 8/10
9/9 [==============================] - 1s 129ms/step - loss: 13.4749
Epoch 9/10
9/9 [==============================] - 1s 130ms/step - loss: 16.5385
Epoch 10/10
9/9 [==============================] - 1s 128ms/step - loss: 10.4329


In [42]:
#preds = model_final.evaluate(x = [X2_test, X_test], y = Y_test)
preds = model_final.evaluate(x = X_test, y = Y_test)

print()
print ("Loss = " + str(preds))

2/2 [==============================] - 0s 24ms/step - loss: 10.6829

Loss = 10.68287467956543
